In [ ]:
# This cell allows the jupyter notebook to take up the space of your full web browser.

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import galXgam
from galXgam.utils import get_rms
from galXgam.utils import gal2equat
from galXgam.utils import catalogue_to_map
from galXgam.utils import map_to_array_of_indices_including_repeats
from galXgam.utils import map_to_catalogue_in_equatorial_coordinates
from galXgam.utils import overdensity_to_mean_counts
from galXgam.utils import sample_galaxy_overdensity_map
from galXgam.utils import overdensity_map_to_density_map
from galXgam.utils import overdensity_map_to_density_map_but_keep_negative_values
from galXgam.utils import overdensity_map_to_density_map_but_keep_negative_values_and_use_rms
from galXgam.utils import read_galaxy_catalogue
from galXgam.utils import read_galaxy_mask
from galXgam.utils import read_and_mask_gamma_ray_map
from galXgam.utils import get_gamma_ray_flux_per_unmasked_pixel
from galXgam.utils import get_gamma_ray_rms_of_unmasked_pixels
from galXgam.utils import compute_treecorr_catalogue_and_pair_counts
from galXgam.utils import compute_xi_and_r_from_treecorr_objects
from galXgam.utils import compute_xi_and_r_from_catalogue_and_random_catalogue
from galXgam.utils import get_random_catalogue_in_equatorial_coordinates_across_an_unmasked_sphere
from galXgam.utils import get_random_catalogue_in_equatorial_coordinates_for_binary_mask
from galXgam.utils import transform_C_ell_to_xi

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from scipy import interpolate
import scipy.stats as st
import treecorr as tr
import numpy as np
import healpy as hp
# import astrotools.healpytools as hpt
import pandas as pd
# import fitsio
# from astropy.io import fits
# import fileinput
import copy
# import camb
# from camb import correlations
# import collections as clt
# import warnings
np.random.seed(12345)

# Enter Directories Here:

In [ ]:
raw_flux_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/raw_photon_exposure_and_foreground_data/new_stuff/flux_wfore"

gamma_ray_map_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/raw_photon_exposure_and_foreground_data/new_stuff/flux_foresub"

gamma_ray_mask_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/raw_photon_exposure_and_foreground_data/new_stuff/masks"

foreground_counts_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/raw_photon_exposure_and_foreground_data/new_stuff/foreground_counts"

foreground_flux_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/raw_photon_exposure_and_foreground_data/new_stuff/foreground_flux"

exposure_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/raw_photon_exposure_and_foreground_data/new_stuff/exposures"

gamma_ray_overdensity_map_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/notebook_scripts_and_outputs/using_raw_data/cross/for_all_z_and_E_bins_new_foreground_subtraction/flask_directory/stefano_cls/flaskpipe/output_old_Cells/sdss_12345"

C_ell_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/notebook_scripts_and_outputs/using_raw_data/cross/for_all_z_and_E_bins_new_foreground_subtraction/flask_directory/stefano_cls/flaskpipe/output_old_C_ells/sdss_12345"

flaskpipe_directory = "/nfs/slac/kipac/fs1/g/des/aresh/Gamma_Ray_x_DES/notebook_scripts_and_outputs/using_raw_data/cross/for_all_z_and_E_bins_new_foreground_subtraction/flask_directory/stefano_cls/flaskpipe"

# Enter Filenames Here:

In [ ]:
raw_flux_filename = raw_flux_directory + "/flux_wfore_12years_9120.1_17378.0_MeV.fits"

gamma_ray_map_filename = gamma_ray_map_directory + "/flux_foresub_12years_9120.1_17378.0_MeV.fits"

gamma_ray_mask_filename = gamma_ray_mask_directory + "/masks_12years_9120.1_17378.0_MeV.fits"

foreground_flux_filename = foreground_flux_directory + "/foreground_flux_12years_9120.1_17378.0_MeV.fits"

foreground_counts_filename = foreground_counts_directory + "/foreground_counts_12years_9120.1_17378.0_MeV.fits"

exposure_filename = exposure_directory + "/exposures_12years_9120.1_17378.0_MeV.fits"

gamma_ray_overdensity_map_filename = gamma_ray_overdensity_map_directory + "/map-f6z5.fits"

# Masked Sampling from Flask Maps

In [ ]:
steradians_per_pixel = (4.0*np.pi)/hp.nside2npix(1024) # this factor needs to be multiplied with the exposure map to convert between flux and photons properly

In [ ]:
#get the exposure map
exposure_map = hp.read_map(exposure_filename)

In [ ]:
#read the data gamma ray flux map, its mask, then mask the map
true_gamma_ray_map = hp.read_map(gamma_ray_map_filename)
gamma_ray_mask_map = hp.read_map(gamma_ray_mask_filename)
true_masked_gamma_ray_map = true_gamma_ray_map * gamma_ray_mask_map

In [ ]:
# convert the data gamma ray flux map to counts by multiplying by the exposure map
true_masked_photon_map = true_masked_gamma_ray_map * exposure_map * steradians_per_pixel * (17378.0 - 9120.1)

In [ ]:
#get the foreground flux map
foreground_flux_map = hp.read_map(foreground_flux_filename)

In [ ]:
#get the foreground counts map
foreground_counts_map = hp.read_map(foreground_counts_filename)

In [ ]:
# add the foreground map to the data map (in counts)
true_masked_photon_plus_foreground_map =  true_masked_photon_map + foreground_counts_map*gamma_ray_mask_map

In [ ]:
raw_flux_map = hp.read_map(raw_flux_filename)

In [ ]:
#make a catalogue of ras and decs by taking every nonzero pixel and finding random ras and decs from within them with weights equal to the pixel flux values
true_ras, true_decs, true_weights = map_to_catalogue_in_equatorial_coordinates(true_masked_gamma_ray_map, "galactic", False)

In [ ]:
#get the gamma ray overdensity map
gamma_ray_overdensity_map = hp.read_map(gamma_ray_overdensity_map_filename)

In [ ]:
print("raw data flux map:")
raw_flux_per_unmasked_pixel = get_gamma_ray_flux_per_unmasked_pixel(raw_flux_map * gamma_ray_mask_map, gamma_ray_mask_map)
print("")
print("")
print("foreground flux map:")
foreground_flux_per_unmasked_pixel = get_gamma_ray_flux_per_unmasked_pixel(foreground_flux_map * gamma_ray_mask_map, gamma_ray_mask_map)
print("")
print("")
print("raw data - foreground flux map:")
gamma_ray_flux_per_unmasked_pixel = raw_flux_per_unmasked_pixel - foreground_flux_per_unmasked_pixel
print("gamma_ray_flux_per_unmasked_pixel: {0}".format(gamma_ray_flux_per_unmasked_pixel))

In [ ]:
#overdensity to density gamma ray flux map
gamma_ray_density_map = overdensity_map_to_density_map_but_keep_negative_values(gamma_ray_overdensity_map, gamma_ray_flux_per_unmasked_pixel)

In [ ]:
#turn the density gamma ray flux map to a density gamma ray counts map
photon_map = gamma_ray_density_map * exposure_map * steradians_per_pixel * (17378.0 - 9120.1)

In [ ]:
#get the foreground counts map
foreground_counts_map = hp.read_map(foreground_counts_filename)

In [ ]:
#get the foreground flux map
foreground_flux_map = hp.read_map(foreground_flux_filename)

In [ ]:
#add the foreground counts map to the density gamma ray counts map
photon_plus_foreground_map =  photon_map + foreground_counts_map

In [ ]:
#Poisson sample
photon_plus_foreground_map = np.where(photon_plus_foreground_map >= 0.0, photon_plus_foreground_map, 0.0)
sampled_photon_plus_foreground_map = np.random.poisson(photon_plus_foreground_map).astype(np.float)

In [ ]:
#convert the Poisson-Sampled Simulated Gamma Ray counts map to a Poisson-Sampled Simulated Gamma Ray flux map
sampled_gamma_ray_plus_foreground_map = sampled_photon_plus_foreground_map * (1.0/exposure_map) * (1.0/steradians_per_pixel) * (1.0/(17378.0 - 9120.1))

In [ ]:
#subtract the foreground
sampled_gamma_ray_map = sampled_gamma_ray_plus_foreground_map - foreground_flux_map

# Compare the histograms of the masked Poisson-Sampled, Foreground-Subtracted Simulated Gamma Ray flux map vs the masked foreground subtracted data counts map

In [ ]:
plt.figure(figsize=(14,9))
h = plt.hist(((sampled_photon_plus_foreground_map-foreground_counts_map)* gamma_ray_mask_map)[np.nonzero((sampled_photon_plus_foreground_map-foreground_counts_map)* gamma_ray_mask_map)], bins=100, label="flask output", histtype="step")
h = plt.hist(true_masked_photon_map[true_masked_photon_map!=0], bins=100, label="data", histtype="step")
plt.legend()
plt.yscale("log")

# Get the input C_ells

In [ ]:
C_ell_galaxy_filename = C_ell_directory + "/sdssCl-f6z5f6z5.dat"
a = pd.read_csv(C_ell_galaxy_filename, sep=" ",header=None) #read the C_ell and ell file
semi_full_ell = a[0].values[0:1024] #get the l values
semi_full_C_ell_original = a[1].values[0:1024] #get the C_ell values at every l
ell = a[0].values[9:1024] #get the l values
C_ell_original = a[1].values[9:1024] #get the C_ell values at every l

# Comparing Input C_ells vs C_ells from Poisson-Sampled Simulated Gamma Ray Flux Map

In [ ]:
gamma_ray_alm_output = hp.map2alm(sampled_gamma_ray_map, lmax=1024)
C_ell_from_flask_map = hp.alm2cl(gamma_ray_alm_output)[10:]

In [ ]:
plt.figure(figsize=(14,7))
plt.scatter(ell, C_ell_original, label="original")
plt.scatter(ell, C_ell_from_flask_map, label="simulated gamma ray flux map (with Poisson sampling)")
plt.xscale("log")
plt.xlabel("l")
plt.ylabel("C_ell")
plt.xlim(8,1200)
plt.ylim(1E-27,1E-25)
plt.title("C_ell, gamma ray auto")
plt.yscale("log")
plt.legend()

# Comparing Input C_ells vs C_ells Raw Overdensity Produced By FLASK

In [ ]:
gamma_ray_alm_output = hp.map2alm(gamma_ray_overdensity_map, lmax=1024)
C_ell_from_flask_map = hp.alm2cl(gamma_ray_alm_output)[10:]

In [ ]:
plt.figure(figsize=(14,7))
plt.scatter(ell, C_ell_original, label="original")
plt.scatter(ell, C_ell_from_flask_map, label="flask map")
plt.xscale("log")
plt.xlabel("l")
plt.ylabel("C_ell")
plt.xlim(8,1200)
plt.ylim(1E-6,1E-1)
plt.title("C_ell, gamma ray auto")
plt.yscale("log")
plt.legend()

# Transforming input C_ells to a CF and comparing that to CF from the Simulated Gamma Ray Map, the Overdensity Map, and the Data (Which is a Gamma Ray Flux Map)

In [ ]:
tck = interpolate.splrep(semi_full_ell, semi_full_C_ell_original) #make interpolator
a_1_final = interpolate.splev([0], tck) #interpolate for C_ell to extend it later
new_ell = [0] + semi_full_ell.tolist() #extend the ell
new_C_ell_original = a_1_final.tolist() + semi_full_C_ell_original.tolist() #extend the C_ell

#make sure neither ell nor C_ell goes beyond 1024 (since that's the nside we're using)
ell = np.array(new_ell[:1025])
C_ell_original = new_C_ell_original[:1025]

In [ ]:
r = np.load(flaskpipe_directory + "/r.npy")

In [ ]:
# take the thetas at which to compute the transform of the input C_ells and change the units from arcmin to cosines
theta_arcmin = copy.deepcopy(r)
theta = np.array(theta_arcmin) / 60.0 #arcmin to degrees
theta_radian = np.radians(theta) #degrees to radians
cosines = np.cos(theta_radian) #radians to cosines

In [ ]:
xi_fourier_from_C_ell_original = transform_C_ell_to_xi(C_ell_original, ell, cosines)

In [ ]:
xi_final = np.load(gamma_ray_overdensity_map_directory+"/xi_f6z5_f6z5.npy")

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(r, xi_fourier_from_C_ell_original, linestyle='-', marker='o', markersize=7, label = "CF from transforming the input C_ells")
plt.plot(r, xi_final, linestyle='-', marker='o', markersize=7, label = "CF measured from simulated gamma ray flux map")
plt.xscale("log")
plt.xlabel("theta [arcmin]")
plt.title("CF, gamma ray auto (scaled to match)")
plt.ylim(bottom=0.0)
plt.legend()